## [ Part 1 gcp Instance creation]

Follow this google link to create Service account key and select json format.
https://cloud.google.com/docs/authentication/getting-started

And set this variable

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path_to_json_file'

### Install google python api client packages

In [ ]:
!pip install --upgrade google-api-python-client

### Importing  gcp libraries and setting up credential

In [ ]:
import argparse
import os
import time
from pprint import pprint
import googleapiclient.discovery
from six.moves import input

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/sathish/gcp_pem.json'
compute = googleapiclient.discovery.build('compute', 'v1')
project="ferrous-weaver-249914"
zone="us-east1-b"

### Data nodes and master node definition

Refer Google page https://cloud.google.com/vpc/docs/vpc to find out default network internal Ipaddress range for your selected region

Ex. us-east1	10.142.0.0/20	10.142.0.1	10.142.0.2 to 10.142.15.253

### Select machine types 

https://cloud.google.com/compute/docs/machine-types

n1-standard-1	1	3.75	128	257	Yes	2
n1-standard-2	2	7.50	128	257	Yes	10
n1-standard-4	4	15	128	257	Yes	10


In [ ]:
machine_size="n1-standard-2"
data_node_instances={'datanode1':{'instance_name':'datanode1','hostname':'datanode1.tanu.com','networkIP':'10.142.15.199','script_template':'cloudera_datanode.sh'},
                   'datanode2':{'instance_name':'datanode2','hostname':'datanode2.tanu.com','networkIP':'10.142.15.200','script_template':'cloudera_datanode.sh'},
                   'datanode3':{'instance_name':'datanode3','hostname':'datanode3.tanu.com','networkIP':'10.142.15.201','script_template':'cloudera_datanode.sh'}}

master_node_instances={'masternode1':{'instance_name':'masternode','hostname':'master.tanu.com','networkIP':'10.142.15.198','script_template':'cloudera_manager.sh'}}


In [ ]:

def generate_data_node_startup_scripts(master_node_name,master_node_ip,script_template):
    
    script = open('new-client-startup-script.sh', 'w') 
    for masternode in master_node_instances:
        networkIP=master_node_instances[masternode]['networkIP']
        hostname=master_node_instances[masternode]['hostname']
        print("echo {networkIP} {hostname}>>/etc/hosts ##adding Masternode host".format(networkIP=networkIP,hostname=hostname),file=script)
    for datanode in data_node_instances:
        networkIP=data_node_instances[datanode]['networkIP']
        hostname=data_node_instances[datanode]['hostname']
        print("echo {networkIP} {hostname}>>/etc/hosts ##adding Data host".format(networkIP=networkIP,hostname=hostname),file=script)
    
    print('''
    #!/bin/bash

    WORK_DIR="/var/tmp/startup_dir"
    
    mkdir $WORK_DIR
    cd $WORK_DIR
    hostname=$(hostname -f)
    #echo "{master_node_ip} {master_node_name}" >>/etc/hosts
    yum -y install git-core net-tools krb5-workstation
    git clone https://github.com/skumarx87/openldap_MIT-Kerberos_installation.git
    cd openldap_MIT-Kerberos_installation
    sed -i "s/idm.tanu.com/{master_node_name}/g" kerberos_ldap_installation.sh
    ./kerberos_ldap_installation.sh client_setup |tee -a kerberos_install.log
    sh {script_template} {master_node_name}|tee -a cm_agent_install.log
    '''.format(master_node_ip=master_node_ip,master_node_name=master_node_name,script_template=script_template),file=script)

    script.close()

def generate_master_node_startup_scripts(master_node_name,script_template):
    script = open('new-server-startup-script.sh', 'w') 
    
    for masternode in master_node_instances:
        networkIP=master_node_instances[masternode]['networkIP']
        hostname=master_node_instances[masternode]['hostname']
        print("echo {networkIP} {hostname}>>/etc/hosts ##adding Masternode host".format(networkIP=networkIP,hostname=hostname),file=script)
    for datanode in data_node_instances:
        networkIP=data_node_instances[datanode]['networkIP']
        hostname=data_node_instances[datanode]['hostname']
        print("echo {networkIP} {hostname}>>/etc/hosts ##adding Data host".format(networkIP=networkIP,hostname=hostname),file=script)
    
    print('''
    #!/bin/bash
    WORK_DIR="/var/tmp/startup_dir"
    mkdir $WORK_DIR
    cd $WORK_DIR
    echo "this is startup script"
    yum -y install git-core net-tools krb5-workstation
    git clone https://github.com/skumarx87/openldap_MIT-Kerberos_installation.git
    cd openldap_MIT-Kerberos_installation
    sed -i "s/idm.tanu.com/{master_node_name}/g" kerberos_ldap_installation.sh
    ./kerberos_ldap_installation.sh server_setup|tee -a kerberos_install.log
    ./kerberos_ldap_installation.sh setup_webserver|tee -a kerberos_install.log
    ./kerberos_ldap_installation.sh client_setup|tee -a kerberos_install.log
    ./kerberos_ldap_installation.sh create_hadoop_users|tee -a kerberos_install.log
    chmod 755 {script_template}
    sh {script_template} |tee -a cm_install.log
    sh cloudera_datanode.sh {master_node_name} |tee -a cm_agent_install.log

    '''.format(master_node_name=master_node_name,script_template=script_template),file=script)
    
    script.close()
    
def list_instances():
        result = compute.instances().list(project=project, zone=zone).execute()
        return result['items'] if 'items' in result else None
        # [END list_instances]

def list_images():
        request = compute.images().list(project=project).execute()
        print(request)
        #while request is not None:
        #       response = request.execute()
        #       print(response)
                #for image in response['items']:
                #       pprint(image)
        #       request = compute.images().list_next(previous_request=request, previous_response=response)
def list_disks():
        request = compute.disks().list(project=project, zone=zone)
        while request is not None:
            response = request.execute()
            for disk in response['items']:
                pprint(disk)
            request = service.disks().list_next(previous_request=request, previous_response=response)
        
        
def get_instance_networkIP(instance):
        request = compute.instances().get(project=project, zone=zone, instance=instance)
        response = request.execute()
        pprint(response)
        host_ip=response['networkInterfaces'][0]['networkIP']
        return host_ip
def get_instance_networkNatIP(instance):
        request = compute.instances().get(project=project, zone=zone, instance=instance)
        response = request.execute()
        #pprint(response)
        nat_ip=response['networkInterfaces'][0]['accessConfigs'][0]['natIP']
        return nat_ip
def get_instance_metadata_fingerprint(instance):
        request = compute.instances().get(project=project, zone=zone, instance=instance)
        response = request.execute()
        pprint(response)
        fingerprint=response['metadata']['fingerprint']
        return fingerprint
    
def delete_instance(instance_name):
    request = compute.instances().delete(project=project, zone=zone, instance=instance_name)
    response = request.execute()
    pprint(response)
    
def create_instance(name,hostname,startup_script,networkIP):
        image_response = compute.images().getFromFamily(project='centos-cloud', family='centos-7').execute()
        source_disk_image = image_response['selfLink']
        #machine_type = "zones/%s/machineTypes/n1-standard-1" % zone
        machine_type = "zones/{zone}/machineTypes/{machine_size}".format(zone=zone,machine_size=machine_size)
        startup_script = open(
                os.path.join(
                        os.path.abspath(''), startup_script), 'r').read()
        config = {
            'name': name,
            'machineType': machine_type,
            'hostname':hostname,
            'disks': [
                {
                    'boot': True,
                    'autoDelete': True,
                    'initializeParams': {
                        'sourceImage': source_disk_image,
                        'diskSizeGb': 20,
                    }
                }
            ],
            # Specify a network interface with NAT to access the public
            # internet.
            'networkInterfaces': [{
                'network': 'global/networks/default',
                'networkIP':networkIP,
                'accessConfigs': [
                    {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
                ]
            }],
            'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script',
                'value': startup_script
            }]
            }
        }
        return compute.instances().insert(project=project,zone=zone,body=config).execute()
            
def insert_filrewall_rule(name,ports):
    firewall_body={
        'name':name,
        'allowed':[
            {
                "IPProtocol":'tcp',
                "ports":ports
            }
        ],
        "sourceRanges": ['0.0.0.0/0']
    }
    request = compute.firewalls().insert(project=project, body=firewall_body)
    response = request.execute()
    pprint(response)
    
def wait_for_operation(operation):
    print('Waiting for operation to finish...')
    while True:
        result = compute.zoneOperations().get(
            project=project,
            zone=zone,
            operation=operation).execute()

        if result['status'] == 'DONE':
            print("done.")
            if 'error' in result:
                raise Exception(result['error'])
            return result

        time.sleep(1)

def Create_hadoop_instaces(wait=True):
        #compute = googleapiclient.discovery.build('compute', 'v1')
        print("Creating Master node")    
                     
        for masternode in master_node_instances:
            inst_name=master_node_instances[masternode]['instance_name']
            host_name=master_node_instances[masternode]['hostname']
            networkIP=master_node_instances[masternode]['networkIP']
            script_template=master_node_instances[masternode]['script_template']
            generate_master_node_startup_scripts(host_name,script_template)
            operation=create_instance(inst_name,host_name,'new-server-startup-script.sh',networkIP)
            print(operation)
            wait_for_operation(operation['name'])

        master_node_inst_name=master_node_instances['masternode1']['instance_name']
        master_node_host_name=master_node_instances['masternode1']['hostname']
        master_node_ip=get_instance_networkIP(master_node_inst_name)
        time.sleep(80) ## wait for master node startup script completion
        for datanode in data_node_instances:
            print("Creating Data Node : {}".format(datanode))
            inst_name=data_node_instances[datanode]['instance_name']
            host_name=data_node_instances[datanode]['hostname']
            networkIP=data_node_instances[datanode]['networkIP']
            script_template=data_node_instances[datanode]['script_template']
            generate_data_node_startup_scripts(master_node_host_name,master_node_ip,script_template)
            operation=create_instance(inst_name,host_name,'new-client-startup-script.sh',networkIP)
            wait_for_operation(operation['name'])
            

def remove_startup_script(wait=True):
    
    for masternode in master_node_instances:
        inst_name=master_node_instances[masternode]['instance_name']
        fingerprint_id=get_instance_metadata_fingerprint(inst_name)
        metadata_body={'fingerprint': fingerprint_id,
              "items": [
            {
              "key": "startup-script",
              "value": "echo startup-script"
            }]
            }
        request = compute.instances().setMetadata(project=project, zone=zone, instance=inst_name, body=metadata_body)
        response = request.execute()
        pprint(response)

    for datanode in data_node_instances:
        inst_name=data_node_instances[datanode]['instance_name']
        fingerprint_id=get_instance_metadata_fingerprint(inst_name)
        metadata_body={'fingerprint': fingerprint_id,
              "items": [
            {
              "key": "startup-script",
              "value": "echo startup-script"
            }]
            }
        request = compute.instances().setMetadata(project=project, zone=zone, instance=inst_name, body=metadata_body)
        response = request.execute()
        pprint(response)

In [ ]:
if __name__ == '__main__':
        Create_hadoop_instaces()
        time.sleep(80)## wait for data node startup script completion
        remove_startup_script()
        #Inserting Firewall rule to allow cloudera manager url port 7180 over the internet
        insert_filrewall_rule("cloudera-manager-url",["7180"])
        #Inserting Firewall rule to allow Namenode url port 9870 over the internet
        insert_filrewall_rule("name-node",["9870"])
        #Inserting Firewall rule to allow Yarn resource manager url port 8088 over the internet
        insert_filrewall_rule("yarn-resource-node",["8088"])
        #Inserting Firewall rule to allow Yarn resource history url port 8088 over the internet
        insert_filrewall_rule("yarn-history-node",["19888"])

## [ Part 2 Hadoop Cluster Creation]
This Section is for creating Hadoop cluster and Cloudera management services

In [ ]:
!pip install cm_client

In [ ]:
from __future__ import print_function
import time
import cm_client
from cm_client.rest import ApiException
from pprint import pprint
import hashlib
import random
from html.parser import HTMLParser
import urllib.request
import urllib
from urllib.parse import urlparse
from itertools import chain
import json
import re

## Update Cloudera Manager URL

In [ ]:
api_host = 'http://35.185.24.52' ##
port = '7180'
api_version = 'v30'

api_url = api_host + ':' + port + '/api/' + api_version
api_client = cm_client.ApiClient(api_url)

cm_client.configuration.username = 'admin'
cm_client.configuration.password = 'admin'
cluster_name='Cluster 8'
cdh_version='CDH6'
hostname='master.tanu.com'

# create an instance of the API class
api_client=cm_client.ApiClient(api_url)

## Cluster Services Mappings

In [ ]:
cdhversion='6.3.2'
cdh_repo_url = 'https://archive.cloudera.com/cdh6/'
## Map the GCP Instance to respective Services like below template ##

Server_mapping={'zookeeper':{'hosts':['datanode1.tanu.com','datanode2.tanu.com','datanode3.tanu.com']},
               'hdfs':{'NAMENODE':{'hosts':['datanode1.tanu.com']},
                      'SECONDARYNAMENODE':{'hosts':['datanode2.tanu.com']},
                      'GATEWAY':{'hosts':['master.tanu.com']},
                      'DATANODE':{'hosts':['datanode1.tanu.com','datanode2.tanu.com','datanode3.tanu.com']}},
               'hive':{'hosts':['datanode1.tanu.com']},
               'yarn':{'hosts':['datanode1.tanu.com','datanode2.tanu.com']},
               'oozie':{'hosts':['datanode1.tanu.com','datanode2.tanu.com']}
              }

### List of CLoudera Mangment Services to be created (Note: had issue with NAVIGATORMETADATASERVER)
MGMT_SERVICES=["EVENTSERVER", "HOSTMONITOR", "ALERTPUBLISHER", "SERVICEMONITOR","ACTIVITYMONITOR"]

#MGMT_SERVICES=["NAVIGATORMETADATASERVER"]

AMON_ROLE_CONFIG = {
   'firehose_database_host':'master.tanu.com' + ":3306",
   'firehose_database_user':'amon',
   'firehose_database_password':'amon',
   'firehose_database_type':'mysql',
   'firehose_database_name':'amon',
   'firehose_heapsize':'314572800'
}

RMAN_ROLE_CONFIG = {
   'headlamp_database_host': 'master.tanu.com'+":7432",
   'headlamp_database_user': 'rman',
   'headlamp_database_password': 'rman',
   'headlamp_database_type': 'mysql',
   'headlamp_database_name': 'rman',
   'headlamp_heapsize': '215964392',
}

NAV_ROLE_CONFIG = {
   'navigator_database_host': 'master.tanu.com'+":7432",
   'navigator_database_user': 'nav',
   'navigator_database_password': 'nav',
   'navigator_database_type': 'mysql',
   'navigator_database_name': 'nav',
   'navigator_heapsize': '215964392',
}


In [ ]:

def manifest_to_dict(manifest_json):
    if manifest_json:
        dir_list = json.load(
            urllib.request.urlopen(manifest_json))['parcels'][0]['parcelName']
        parcel_part = re.match(r"^(.*?)-(.*)-(.*?)$", dir_list).groups()
        return {'product': str(parcel_part[0]).upper(), 'version': str(parcel_part[1]).lower()}
    else:
        raise Exception("Invalid manifest.json")

class LinkParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.links = iter([])

    def reset(self):
        HTMLParser.reset(self)

    def handle_starttag(self, tag, attrs):
        # Only parse the 'anchor' tag.

        if tag == "a":
            for name, value in attrs:
                if '6' in value:
                    self.links = chain(self.links, [value.strip('/')])
                    
def _check_parcel_stage(product, version, expected_stage):
    parcel_instance = cm_client.ParcelResourceApi(api_client)
    while True:
        cdh_parcel = parcel_instance.read_parcel(cluster_name=cluster_name,
                                                 product=product, version=version)
        if cdh_parcel.stage in expected_stage:
            break
        if cdh_parcel.state.errors:
            raise Exception(str(cdh_parcel.state.errors))
        time.sleep(1)
                    
def parcel_action(product, version, function, expected_stage, action_description):
    parcel_instance = cm_client.ParcelResourceApi(api_client)
    print("%s [%s-%s]" % (action_description, product, version))
    cdh_parcel = parcel_instance
    cmd = getattr(cdh_parcel, function)(cluster_name=cluster_name, product=product, version=version)
    if not cmd.success:
        print("ERROR: %s failed!" % action_description)
        exit(0)
    return _check_parcel_stage(product, version, expected_stage)

def install_parcel():
    parser = LinkParser()
    parser.feed(str(urllib.request.urlopen(cdh_repo_url).read()))
    cdh_version = [x for x in parser.links if cdhversion.strip() in x]
    cdh_parcel = manifest_to_dict("%s%s/parcels/manifest.json" % (cdh_repo_url, cdh_version[0]))
    # Deploy Parcel > Download> Distribute> Activate
    product=cdh_parcel['product']
    version=cdh_parcel['version']
    parcel_instance = cm_client.ParcelResourceApi(api_client)
    try:
        # A synchronous command that starts the parcel download.
        cluster_parcel = parcel_instance.read_parcel(cluster_name,
                                                     product=product,
                                                     version=version)
        print("Parcel current Stage: [%s]" % cluster_parcel.stage)
        if "ACTIVATED" not in cluster_parcel.stage:
            parcel_action(product=product, version=version, function="start_download_command",
                          expected_stage=['DOWNLOADED'],
                          action_description="Download Parcel")
            parcel_action(product=product, version=version, function="start_distribution_command",
                          expected_stage=['DISTRIBUTED'],
                          action_description="Distribute Parcel")
            parcel_action(product=product, version=version, function="activate_command",
                          expected_stage=['ACTIVATED'], action_description="Activate Parcel")
        elif 'ACTIVATED' == cluster_parcel.stage:
            parcel_action(product=product, version=version,
                          function="deactivate_command", expected_stage=['DISTRIBUTED'],
                          action_description="Deactivate Parcel")
            parcel_action(product=product, version=version,
                          function="start_removal_of_distribution", expected_stage=['DOWNLOADED'],
                          action_description="Un-Distribute Parcel")
    except ApiException as e:
        print("Exception when calling ParcelResourceApi->start_download_command: %s\n" % e)
            
    
    #parcel_instance = cm_client.ParcelResourceApi(api_client)
    #print("%s [%s-%s]" % (action_description, product, version))
    
def create_cluster():
    hosts=[]
    _hosts = []
    [hosts.append(data_node_instances[key]['hostname']) for key in data_node_instances.keys()]
    [hosts.append(master_node_instances[key]['hostname']) for key in master_node_instances.keys()]
    [_hosts.append({'hostname': host, 'hostId':getattr(get_host_resource(host),'host_id', None)}) for host in hosts]
    cluster1={'items':[
        {'name': cluster_name,
         "version": cdh_version,
        }
        ]}
    try:
        api_instance = cm_client.ClustersResourceApi(api_client)
        api_instance.create_clusters(body=cluster1);
        api_instance.add_hosts(cluster_name,body=cm_client.ApiHostRefList(_hosts))
    #print(cluster)
    #cluster.add_hosts(HOSTS_LIST)
    #pprint(api_response)
    except ApiException as e:
        print("Exception when calling ClustersResourceApi->create_clusters: %s\n" % e)
def get_host_resource(hostname):
    api_instance = cm_client.HostsResourceApi(api_client)
    #print(api_instance.read_hosts(view='summary'))
    try:
        api_host_response = [x for x in api_instance.read_hosts(view='summary').items
                             if hostname == x.hostname]
    except ApiException as e:
        print("Exception when calling HostsResourceApi->read_hosts: %s\n" % e)
    #print(api_host_response)
    return api_host_response[0]

def add_mgmt_service():
    amon_role_name = "ACTIVITYMONITOR"
    service_name = 'mgmt'
    service_type = 'MGMT'.upper()
    hostname='master.tanu.com'
    host_id = getattr(get_host_resource(hostname),'host_id', None)
    services_instance = cm_client.MgmtServiceResourceApi(api_client)
    mgmt_role_instance = cm_client.MgmtRolesResourceApi(api_client)
    rcg_instance = cm_client.MgmtRoleConfigGroupsResourceApi(api_client)
    print(mgmt_role_instance)
    try:
        print("Create a CMS service and its associated roles.")
        services_instance.setup_cms(body=cm_client.ApiService(name=service_name,
                                                              type=service_type,
                                                              display_name='Cloudera Management Service'))
        role_list = []
        for role_type in MGMT_SERVICES:
            role_name = "%s-%s-%s" % (service_name, role_type, hashlib.md5(hostname.encode('utf-8')).hexdigest())
            #print(host_id)
            role_list.append({"name": role_name, "type": role_type, "hostRef": {"hostId": host_id}})
        #print(role_list)
        body = cm_client.ApiRoleList(role_list)
        api_response = mgmt_role_instance.create_roles(body=body)
        api_response = rcg_instance.read_role_config_groups()
        for rcg in api_response.items:
            print(rcg)
            if rcg.role_type == 'ACTIVITYMONITOR':
                activity_config=[]
                tmp={activity_config.append({'name':key,'value':value}) for key,value in AMON_ROLE_CONFIG.items()}
                print(tmp)
                rcg_instance.update_config(rcg.name, message=None,
                                           body=cm_client.ApiConfigList(
                                               activity_config
                                           ))                                       
                                            
            if rcg.role_type == 'REPORTMANAGER':
                rmon_config = []
                tmp={rmon_config.append({'name':key,'value':value}) for key,value in RMAN_ROLE_CONFIG.items()}
                rcg_instance.update_config(rcg.name, message=None,
                                           body=cm_client.ApiConfigList([
                                               rmon_config
                                            ]))
            '''
            if rcg.role_type == 'NAVIGATOR':
                nav_config = []
                tmp={nav_config.append({'name':key,'value':value}) for key,value in NAV_ROLE_CONFIG.items()}
                rcg_instance.update_config(rcg.name, message=None,
                                           body=cm_client.ApiConfigList([
                                               nav_config
                                            ]))
            '''
        services_instance.start_command()
    except ApiException as e:
        print("Exception: %s\n" % e)
def create_roles(service_name, role_type, hostname):
    roles_instance = cm_client.RolesResourceApi(api_client)
    host_id = getattr(get_host_resource(hostname), 'host_id', None)
    first_part = service_name[:4] + hashlib.md5(service_name).hexdigest()[:8] \
        if len(role_type) > 24 else service_name
    role_name = "-".join([first_part, role_type, hashlib.md5(hostname.encode('utf-8')).hexdigest()])[:64]
    api_response = roles_instance.create_roles(
        cluster_name, service_name, body=cm_client.ApiRoleList(
            [{'name': role_name,
              'type': role_type,
              'hostRef': {'hostId': host_id}
              }])
    )
    print(api_response)
    return api_response.items[0]

def get_service_type(name):
    """
    Returns service based on service type name
    :param name:
    :return:
    """

    services_instance = cm_client.ServicesResourceApi(api_client)
    try:
        service = [x for x in services_instance.read_services(cluster_name).items if name.upper() == x.type][0]
    except IndexError:
        service = None

    return service


def dependencies_for(service_name):
    """
    Utility function returns dict of service dependencies
    :return:
    """
    service_config = []
    config_types = {"hue_webhdfs": ['NAMENODE', 'HTTPFS'], "hdfs_service": "HDFS", "sentry_service": "SENTRY",
                    "zookeeper_service": "ZOOKEEPER", "hbase_service": "HBASE",
                    "hue_hbase_thrift": "HBASETHRIFTSERVER", "solr_service": "SOLR",
                    "hive_service": "HIVE", "sqoop_service": "SQOOP",
                    "impala_service": "IMPALA", "oozie_service": "OOZIE",
                    "mapreduce_yarn_service": ['MAPREDUCE', 'YARN'], "yarn_service": "YARN"}

    dependency_list = []
    # get required service config
    services_instance = cm_client.ServicesResourceApi(api_client)
    service_conf = services_instance.read_service_config(cluster_name, service_name, view='full')

    for name, required in [(config.name, config.required) for config in service_conf.items if config.required]:
        if required:
            dependency_list.append(name)

    service = services_instance.read_service(cluster_name, service_name)
    # print service.type

    # Extended dependence list, adding the optional ones as well
    if service.type == 'HUE':
        dependency_list.extend(['hbase_service', 'solr_service', 'sqoop_service',
                                'impala_service', 'hue_hbase_thrift', 'hive_service'])
    if service.type in ['HIVE', 'HDFS', 'HUE', 'OOZIE', 'MAPREDUCE', 'YARN', 'ACCUMULO16']:
        dependency_list.append('zookeeper_service')
    if service.type in ['HIVE']:
        dependency_list.append('sentry_service')
    if service.type == 'OOZIE':
        dependency_list.append('hive_service')
    if service.type in ['FLUME', 'IMPALA']:
        dependency_list.append('hbase_service')
    if service.type in ['FLUME', 'SPARK', 'SENTRY', 'ACCUMULO16']:
        dependency_list.append('hdfs_service')
    if service.type == 'FLUME':
        dependency_list.append('solr_service')
        
    role_instance = cm_client.RolesResourceApi(api_client)
    for key in dependency_list:
        if key == "hue_webhdfs":
            service_name = get_service_type('HDFS')

            if service_name is not None:
                service_roles = role_instance.read_roles(cluster_name, service_name.name,
                                                            filter='', view='full')
                service_config.append({'name': key,
                                        'value': [role for role in service_roles.items
                                                    if 'NAMENODE' == role.type][0].name})

            if [role for role in service_roles.items if 'HTTPFS' == role.type][0]:
                service_config.remove({'name': key,
                                        'value': [role for role in service_roles.items
                                                    if 'NAMENODE' == role.type][0].name})
                service_config.append({'name': key,
                                        'value': [role for role in service_roles.items
                                                    if 'HTTPFS' == role.type][0].name})

        elif key == "mapreduce_yarn_service":
            for _type in config_types[key]:
                if get_service_type(_type) is not None:
                    service_config.append({'name': key, 'value': get_service_type(_type).name})
                    # prefer YARN over MAPREDUCE
                if get_service_type(_type) is not None and _type == 'YARN':
                    service_config.remove({'name': key, 'value': get_service_type(_type).name})
                    service_config.append({'name': key, 'value': get_service_type(_type).name})

        elif key == "hue_hbase_thrift":
            service_name = get_service_type('HBASE')
            if service_name is not None:
                service_roles = role_instance.read_roles(cluster_name, service_name.name,
                                                            filter='', view='full')
                service_config.append({'name': key, 'value':
                    [role for role in service_roles.items if config_types[key] == role.type][0].name})
        else:
            if get_service_type(config_types[key]) is not None:
                service_config.append({'name': key, 'value': get_service_type(config_types[key]).name})

    return service_config


def add_zookeeper():
    service_name = 'zookeeper'
    servers_list=Server_mapping[service_name]['hosts']
    service_type = 'ZOOKEEPER'.upper()
    #for hostname in random.sample(servers_list, 3 if len(servers_list) >= 3 else 1):
    #    print(hostname)
    services_instance = cm_client.ServicesResourceApi(api_client)
    rcg_instance = cm_client.RoleConfigGroupsResourceApi(api_client)
    
    try:
        print("Create a %s service and its associated roles." % service_name)
        services_instance.create_services(cluster_name,
                                          body=cm_client.ApiServiceList(
                                              [{'name': service_name,
                                                'type': service_type}]))
        services_instance.update_service_config(cluster_name, service_name, message=None,
                                                body=cm_client.ApiServiceConfig([
                                                    {'name': 'zookeeper_datadir_autocreate', 'value': False},
                                                    ]))
        # Updates the config for the given role config group.
        api_response = rcg_instance.read_role_config_groups(cluster_name, service_name)
        for rcg in api_response.items:
            if rcg.role_type == 'SERVER':
                rcg_instance.update_config(
                    cluster_name, rcg.name, service_name, message=None,
                    body=cm_client.ApiConfigList([
                        {'name': 'maxClientCnxns', 'value': '1024'},
                        {'name': 'zookeeper_server_java_heapsize', 'value': '492830720'},
                        {'name': 'maxSessionTimeout', 'value': '60000'}
                    ])
                )
                # Create new roles in a given service.
                for hostname in random.sample(servers_list, 3 if len(servers_list) >= 3 else 1):
                    role = create_roles(service_name=service_name, role_type=rcg.role_type,hostname=hostname)
                                    
    except ApiException as e:
        print("Exception: %s\n" % e)

def add_hdfs():
    service_name = 'hdfs'
    #servers_list=Server_mapping[service_name]['hosts']
    service_type = 'HDFS'.upper()
    #for hostname in random.sample(servers_list, 3 if len(servers_list) >= 3 else 1):
    #    print(hostname)
    services_instance = cm_client.ServicesResourceApi(api_client)
    rcg_instance = cm_client.RoleConfigGroupsResourceApi(api_client)
    
    try:
        print("Create a %s service and its associated roles." % service_name)
        services_instance.create_services(cluster_name,
                                          body=cm_client.ApiServiceList(
                                              [{'name': service_name,
                                                'type': service_type}]))
        service_config = [{'name': 'hdfs_verify_ec_with_topology_enabled', 'value': 'false'},
                          {'name': 'dfs_replication', 'value': '1'},]
        for kv in dependencies_for(service_name=service_name):
            service_config.append(kv)
        # Update hbase service dependency
        services_instance.update_service_config(cluster_name, service_name, message=None,
                                                body=cm_client.ApiServiceConfig(service_config))
        # Updates the config for the given role config group.
        api_response = rcg_instance.read_role_config_groups(cluster_name, service_name)
        for rcg in api_response.items:
            try:
                servers_list=Server_mapping[service_name][rcg.role_type]['hosts']
            except KeyError:
                print("Key Error ?????????? {}".format(rcg.role_type))
                continue
            print("servers list !!!!!!!!!!!!!!")
            print(servers_list)
         
            if rcg.role_type == 'NAMENODE':
                rcg_instance.update_config(
                    cluster_name, rcg.name, service_name, message=None,
                    body=cm_client.ApiConfigList([
                        {'name': 'dfs_name_dir_list', 'value': '/data/dfs/nn'},
                        {'name': 'dfs_namenode_handler_count', 'value': '30'},
                        {'name': 'dfs_namenode_service_handler_count', 'value': '30'},
                        {'name': 'dfs_namenode_servicerpc_address', 'value': '8022'}
                    ])
                )
                    # Create new roles in a given service.
                for hostname in servers_list:
                    role = create_roles(service_name=service_name,
                                        role_type=rcg.role_type,
                                        hostname=hostname)
                    
            if rcg.role_type == 'SECONDARYNAMENODE':
                rcg_instance.update_config(
                    cluster_name, rcg.name, service_name, message=None,
                    body=cm_client.ApiConfigList([
                        {'name': 'fs_checkpoint_dir_list', 'value': '/data/dfs/snn'},
                    ])
                )
                    # Create new roles in a given service.
                for hostname in servers_list:
                    role = create_roles(service_name=service_name,
                                        role_type=rcg.role_type,
                                        hostname=hostname)
                    
            if rcg.role_type == 'DATANODE':
                rcg_instance.update_config(
                    cluster_name, rcg.name, service_name, message=None,
                    body=cm_client.ApiConfigList([
                        {'name': 'datanode_java_heapsize', 'value': '127926272'},
                        {'name': 'dfs_data_dir_list', 'value': '/data/dfs/dn'},
                        {'name': 'dfs_datanode_data_dir_perm', 'value': '755'},
                        {'name': 'dfs_datanode_du_reserved', 'value': '3218866585'},
                        {'name': 'dfs_datanode_failed_volumes_tolerated', 'value': '0'},
                        {'name': 'dfs_datanode_max_locked_memory', 'value': '316669952'},
                    ])
                )
                    # Create new roles in a given service.
                for hostname in servers_list:
                    role = create_roles(service_name=service_name,
                                        role_type=rcg.role_type,
                                        hostname=hostname)
    except ApiException as e:
        print("Exception: %s\n" % e)
        
if __name__ == '__main__':
    create_cluster();
    add_mgmt_service()
    install_parcel()
    add_zookeeper()
    add_hdfs()

### Uncomment to delete all hadoop compute instances

In [ ]:
#cm_client.ApiConfigList([RMAN_ROLE_CONFIG])
import json
r=json.dumps(AMON_ROLE_CONFIG)
loaded_r = json.loads(r)
#q=[]
#for key in AMON_ROLE_CONFIG:
#    q.append({"name":key,"value":AMON_ROLE_CONFIG[key]})
#print(json.dumps(q))

#x={key: value for key, value in AMON_ROLE_CONFIG}
#values = map(lambda key: AMON_ROLE_CONFIG[key], AMON_ROLE_CONFIG.keys())
#print(values)
u=[]
x={u.append({'name':key,'value':i}) for key,i in AMON_ROLE_CONFIG.items()}
print(json.dumps(u))

In [ ]:
def delete_hadoop_instaces(wait=True):
    for masternode in master_node_instances:
        inst_name=master_node_instances[masternode]['instance_name']
        delete_instance(inst_name)

    for datanode in data_node_instances:
        inst_name=data_node_instances[datanode]['instance_name']
        delete_instance(inst_name)

        
delete_hadoop_instaces()

In [ ]:
def get_host_entry_details():
    for masternode in master_node_instances:
        inst_name=master_node_instances[masternode]['instance_name']
        hostname=master_node_instances[masternode]['hostname']
        nat_ip=get_instance_networkNatIP(inst_name)
        print("{nat_ip} {hostname}".format(nat_ip=nat_ip,hostname=hostname))

    for datanode in data_node_instances:
        inst_name=data_node_instances[datanode]['instance_name']
        hostname=data_node_instances[datanode]['hostname']
        nat_ip=get_instance_networkNatIP(inst_name)
        print("{nat_ip} {hostname}".format(nat_ip=nat_ip,hostname=hostname))

get_host_entry_details()